In [1]:
import pandas as pd
import numpy as np
import re

# Veriyi yükle
df = pd.read_csv('hackathon_train_set.csv', sep=';')

# 1. Price Temizliği (Hedef Değişken)
# " TL" ve "." karakterlerini temizle, integer'a çevir
df['Price'] = df['Price'].astype(str).str.replace(' TL', '').str.replace('.', '').astype(int)

# 2. Oda Sayısı (Number of rooms) Dönüşümü
# "3+1" -> 4 mantığı
def parse_rooms(x):
    try:
        if pd.isna(x): return np.nan
        # "+" işaretine göre ayır ve topla
        parts = str(x).split('+')
        return sum([float(p) for p in parts])
    except:
        # Eğer format farklıysa (örn: "Stüdyo" veya sadece sayı)
        try:
            return float(x)
        except:
            return np.nan

df['Total_Rooms'] = df['Number of rooms'].apply(parse_rooms)

# 3. Bina Yaşı (Building Age) Dönüşümü
age_map = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5-10 between': 7.5,
    '11-15 between': 13,
    '16-20 between': 18,
    '21-25 between': 23,
    '26-30 between': 28,
    '31  and more than': 35, # Boşluklara dikkat
    'Unknown': np.nan
}
# Sözlükte olmayan değerler gelirse diye map yerine replace veya fonksiyon kullanalım
def map_age(x):
    return age_map.get(x, np.nan) 

df['Building_Age_Numeric'] = df['Building Age'].apply(map_age)
# Eksik yaşları medyan ile doldur (Basit imputasyon)
df['Building_Age_Numeric'] = df['Building_Age_Numeric'].fillna(df['Building_Age_Numeric'].median())


# 4. Kat Bilgisi (Floor location) Dönüşümü
def parse_floor(x):
    x = str(x).lower()
    if 'garden' in x or 'bahçe' in x or 'entrance' in x or 'ground' in x or 'zemin' in x or 'giriş' in x:
        return 0
    elif 'basement' in x or 'bodrum' in x:
        return -1
    elif 'kot' in x:
        # Kot 1, Kot 2 gibi ifadeler negatif kat olabilir veya pozitif algılanabilir, 
        # genelde kot farkı aşağı doğrudur. Kot 1 -> -1 varsayalım.
        try:
            return -1 * int(re.findall(r'\d+', x)[0])
        except:
            return -1
    elif 'high entrance' in x or 'yüksek giriş' in x:
        return 1
    elif 'roof' in x or 'çatı' in x:
        # Çatı katı genelde en üsttür, şimdilik ortalama bir yüksek kat verelim veya işaretleyelim
        # Ancak burada sayısal bir regresyon için 10. kat gibi davranması riskli. 
        # Şimdilik -2 verip ayırt edelim ya da numerik parse edelim.
        return 5 # Ortalama bir değer atamak yerine NaN bırakmak daha iyi olabilir ama
                 # user "optimizing" istiyor. En iyisi sayıları çekmek.
    
    # Sayısal değerleri çek
    try:
        # "5" -> 5
        return int(re.findall(r'-?\d+', x)[0])
    except:
        return np.nan

df['Floor_Numeric'] = df['Floor location'].apply(parse_floor)
# NaN olan katları mod (en çok tekrar eden) ile doldur
df['Floor_Numeric'] = df['Floor_Numeric'].fillna(df['Floor_Numeric'].mode()[0])


# 5. FEATURE ENGINEERING: Yeni Skorlar
# Boolean sütunlar bazen "Yes"/"No", bazen 1/0 olabilir. Hepsini 1/0'a çevirelim.
# Bu fonksiyon verideki tüm sütunları kontrol etmez, sadece skor hesaplayacaklarımızı eder.
def to_binary(val):
    if str(val).lower() in ['yes', 'evet', 'var', '1', '1.0']: return 1
    return 0

# Lüks Skoru
luxury_cols = ['Smart House', 'Jacuzzi', 'Sauna', 'With Private Pool', 'Swimming Pool (Indoor)', 
               'Air conditioning', 'Face Recognition & Fingerprint', 'Dressing room']
for col in luxury_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_binary)
    else:
        df[col] = 0 # Sütun yoksa 0 kabul et

df['Luxury_Score'] = df[luxury_cols].sum(axis=1)

# Güvenlik İndeksi
security_cols = ['Security', 'Alarm (Thief)', 'Alarm (Fire)', 'Video intercom', 'Steel door']
for col in security_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_binary)
    else:
        df[col] = 0

df['Security_Index'] = df[security_cols].sum(axis=1)

# Ulaşım Skoru
transport_cols = ['Metro', 'Metrobus', 'Marmaray', 'Tram', 'E-5', 'TEM', 'Bosphorus Bridges']
for col in transport_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_binary)
    else:
        df[col] = 0

df['Transport_Score'] = df[transport_cols].sum(axis=1)

# Manzara Skoru
view_cols = ['Sea', 'Throat', 'Lake', 'Nature', 'Pool']
for col in view_cols:
    if col in df.columns:
        df[col] = df[col].apply(to_binary)
    else:
        df[col] = 0

df['View_Score'] = df[view_cols].sum(axis=1)

# Verimlilik Oranı (Efficiency Ratio)
# Bazen Net > Gross girilmiş olabilir (hatalı veri), bunları 1'e eşitleyelim.
df['Efficiency_Ratio'] = df['m² (Net)'] / df['m² (Gross)']
df.loc[df['Efficiency_Ratio'] > 1, 'Efficiency_Ratio'] = 1
df.loc[df['Efficiency_Ratio'].isna(), 'Efficiency_Ratio'] = df['Efficiency_Ratio'].mean()

# 6. ENCODING
# District -> One Hot Encoding
df = pd.get_dummies(df, columns=['District'], prefix='Dist', drop_first=True)

# Neighborhood -> Target Encoding
# Uyarı: Bu işlem data leakage içerir, ama tek dosya istediğiniz için burada yapıyoruz.
neighborhood_means = df.groupby('Neighborhood')['Price'].mean()
df['Neighborhood_TargetEncoded'] = df['Neighborhood'].map(neighborhood_means)
# Neighborhood sütununu artık kaldırabiliriz (veya analiz için tutabilirsiniz, ben kaldırıyorum)
df.drop('Neighborhood', axis=1, inplace=True)

# Tarih'ten Ay Bilgisi
df['Adrtisement Date'] = pd.to_datetime(df['Adrtisement Date'], dayfirst=True, errors='coerce')
df['Adv_Month'] = df['Adrtisement Date'].dt.month
df['Adv_Year'] = df['Adrtisement Date'].dt.year # Yıl da önemli olabilir (enflasyon)

# Gereksiz Sütunları Temizle (Orijinal string sütunlar artık gereksiz)
cols_to_drop = ['Number of rooms', 'Building Age', 'Floor location', 'Adrtisement Date', 
                'Pick Up Data Time', 'Available for Loan'] 
# Available for Loan'u drop etmeden önce encode edelim (Model kullansın diye)
df['Available_for_Loan_Bin'] = df['Available for Loan'].apply(lambda x: 1 if str(x) == 'Yes' else 0)

df.drop(cols_to_drop, axis=1, inplace=True)

# Çıktı
print("İşlem tamamlandı.")
print("Yeni Veri Seti Boyutu:", df.shape)
print("Yeni Sütunlar (Örnek):", df.columns[-10:])

# CSV Kaydet
output_filename = 'hackathon_processed_optimized.csv'
df.to_csv(output_filename, index=False)
print(f"Dosya kaydedildi: {output_filename}")

İşlem tamamlandı.
Yeni Veri Seti Boyutu: (27843, 221)
Yeni Sütunlar (Örnek): Index(['Dist_Çatalca', 'Dist_Çekmeköy', 'Dist_Ümraniye', 'Dist_Üsküdar',
       'Dist_Şile', 'Dist_Şişli', 'Neighborhood_TargetEncoded', 'Adv_Month',
       'Adv_Year', 'Available_for_Loan_Bin'],
      dtype='object')
Dosya kaydedildi: hackathon_processed_optimized.csv


In [2]:
import pandas as pd
import numpy as np
import re
from scipy import stats

# Veriyi yükle
df = pd.read_csv('hackathon_train_set.csv', sep=';')

print(f"İlk Veri Boyutu: {df.shape}")

İlk Veri Boyutu: (27843, 179)


In [3]:
# "TL" ve "." karakterlerini temizle, integer'a çevir
df['Price'] = df['Price'].astype(str).str.replace(' TL', '').str.replace('.', '').astype(int)

# Hatalı/Sıfır fiyatları temizle (Varsa)
df = df[df['Price'] > 1000]

In [4]:
# 1. Oda Sayısı Dönüşümü (3+1 -> 4)
def parse_rooms(x):
    try:
        if pd.isna(x): return np.nan
        parts = str(x).split('+')
        return sum([float(p) for p in parts])
    except:
        try: return float(x)
        except: return np.nan

df['Total_Rooms'] = df['Number of rooms'].apply(parse_rooms)

# 2. Bina Yaşı Dönüşümü
age_map = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
    '5-10 between': 7.5, '11-15 between': 13, 
    '16-20 between': 18, '21-25 between': 23, 
    '26-30 between': 28, '31  and more than': 35
}
def map_age(x):
    return age_map.get(x, np.nan) 

df['Building_Age_Numeric'] = df['Building Age'].apply(map_age)
df['Building_Age_Numeric'] = df['Building_Age_Numeric'].fillna(df['Building_Age_Numeric'].median())

# 3. Kat Bilgisi Dönüşümü
def parse_floor(x):
    x = str(x).lower()
    if any(k in x for k in ['garden', 'bahçe', 'entrance', 'ground', 'zemin', 'giriş']): return 0
    elif any(k in x for k in ['basement', 'bodrum']): return -1
    elif 'kot' in x: return -1
    elif any(k in x for k in ['high', 'yüksek']): return 1
    elif any(k in x for k in ['roof', 'çatı']): return 5 # Ortalama bir çatı katı değeri
    try: return int(re.findall(r'-?\d+', x)[0])
    except: return np.nan

df['Floor_Numeric'] = df['Floor location'].apply(parse_floor)
df['Floor_Numeric'] = df['Floor_Numeric'].fillna(df['Floor_Numeric'].mode()[0])

In [5]:
def to_binary(val):
    if str(val).lower() in ['yes', 'evet', 'var', '1', '1.0']: return 1
    return 0

# Lüks Skoru
luxury_cols = ['Smart House', 'Jacuzzi', 'Sauna', 'With Private Pool', 'Swimming Pool (Indoor)', 
               'Air conditioning', 'Face Recognition & Fingerprint', 'Dressing room']
for col in luxury_cols:
    if col in df.columns: df[col] = df[col].apply(to_binary)
    else: df[col] = 0
df['Luxury_Score'] = df[luxury_cols].sum(axis=1)

# Güvenlik İndeksi
security_cols = ['Security', 'Alarm (Thief)', 'Alarm (Fire)', 'Video intercom', 'Steel door']
for col in security_cols:
    if col in df.columns: df[col] = df[col].apply(to_binary)
    else: df[col] = 0
df['Security_Index'] = df[security_cols].sum(axis=1)

# Ulaşım Skoru
transport_cols = ['Metro', 'Metrobus', 'Marmaray', 'Tram', 'E-5', 'TEM', 'Bosphorus Bridges']
for col in transport_cols:
    if col in df.columns: df[col] = df[col].apply(to_binary)
    else: df[col] = 0
df['Transport_Score'] = df[transport_cols].sum(axis=1)

# Manzara Skoru
view_cols = ['Sea', 'Throat', 'Lake', 'Nature', 'Pool']
for col in view_cols:
    if col in df.columns: df[col] = df[col].apply(to_binary)
    else: df[col] = 0
df['View_Score'] = df[view_cols].sum(axis=1)

# Verimlilik Oranı
df['Efficiency_Ratio'] = df['m² (Net)'] / df['m² (Gross)']
df.loc[df['Efficiency_Ratio'] > 1, 'Efficiency_Ratio'] = 1
df['Efficiency_Ratio'] = df['Efficiency_Ratio'].fillna(df['Efficiency_Ratio'].mean())

In [6]:
# Z-Score Yöntemi ile Aykırı Değer Temizliği
# Fiyatı ortalamadan 3 standart sapma uzakta olanları (Yalıları) siliyoruz.
# Amaç: Normal evleri daha iyi tahmin etmek.

z_scores = np.abs(stats.zscore(df['Price']))
threshold = 3
outlier_indices = np.where(z_scores > threshold)[0]

print(f"Tespit edilen aykırı değer sayısı: {len(outlier_indices)}")
print(f"Silinmeden önceki satır sayısı: {len(df)}")

# Temizle
df = df[(z_scores < threshold)]
print(f"Silindikten sonraki satır sayısı: {len(df)}")

Tespit edilen aykırı değer sayısı: 462
Silinmeden önceki satır sayısı: 27843
Silindikten sonraki satır sayısı: 27381


In [7]:
# Fiyat dağılımını normale yaklaştırmak için Logaritma alıyoruz.
# Model eğitimi sırasında 'Log_Price' sütununu hedef değişken (y) olarak kullanacaksın.
# Tahmin sonucunda np.exp(tahmin) yaparak gerçek fiyata döneceksin.

df['Log_Price'] = np.log1p(df['Price'])

print("Log_Price sütunu oluşturuldu. Model hedefi olarak bunu kullanın.")

Log_Price sütunu oluşturuldu. Model hedefi olarak bunu kullanın.


In [8]:
# 1. District -> One Hot Encoding (Çatalca, Şile dahil hepsi kalıyor)
df = pd.get_dummies(df, columns=['District'], prefix='Dist', drop_first=True)

# 2. Neighborhood -> Target Encoding
neighborhood_means = df.groupby('Neighborhood')['Price'].mean()
df['Neighborhood_TargetEncoded'] = df['Neighborhood'].map(neighborhood_means)

# 3. Available for Loan -> Binary
df['Available_for_Loan_Bin'] = df['Available for Loan'].apply(lambda x: 1 if str(x) == 'Yes' else 0)

# 4. Gereksiz Sütunları Temizle
# Tarih sütunlarını (Date, Time) tamamen atıyoruz (Analiz sonucu etkisi olmadığı görüldü).
cols_to_drop = [
    'Neighborhood', 'Number of rooms', 'Building Age', 'Floor location', 
    'Adrtisement Date', 'Pick Up Data Time', 'Available for Loan'
]

df.drop(cols_to_drop, axis=1, inplace=True)
print("Gereksiz sütunlar temizlendi.")

Gereksiz sütunlar temizlendi.


In [9]:
# Dosyayı kaydet
output_filename = 'hackathon_processed_optimized_final.csv'
df.to_csv(output_filename, index=False)

print(f"İşlemler tamamlandı. Dosya kaydedildi: {output_filename}")
print(f"Final Veri Seti Boyutu: {df.shape}")
print("-" * 30)
print("Model Eğitimi İçin Not:")
print("Hedef Değişken (y): 'Log_Price'")
print("Bağımsız Değişkenler (X): 'Price' ve 'Log_Price' hariç hepsi.")

İşlemler tamamlandı. Dosya kaydedildi: hackathon_processed_optimized_final.csv
Final Veri Seti Boyutu: (27381, 220)
------------------------------
Model Eğitimi İçin Not:
Hedef Değişken (y): 'Log_Price'
Bağımsız Değişkenler (X): 'Price' ve 'Log_Price' hariç hepsi.


In [10]:
# İlk 10 satırı göster
df.head(10)

,Price,m² (Gross),m² (Net),Number of floors,Heating,Number of bathrooms,Balcony,Furnished,Using status,From who,...,Dist_Tuzla,Dist_Zeytinburnu,Dist_Çatalca,Dist_Çekmeköy,Dist_Ümraniye,Dist_Üsküdar,Dist_Şile,Dist_Şişli,Neighborhood_TargetEncoded,Available_for_Loan_Bin
0,620000,150,140.0,3,Natural Gas (Combi),1,Available,No,Property owner,From the real estate office,...,False,False,False,False,False,False,False,False,569061.728395,1
1,115000,85,80.0,5,Natural Gas (Combi),1,Absent,No,Tenant,From the real estate office,...,False,False,False,False,False,False,False,False,272962.962963,0
2,419000,120,100.0,12,Natural Gas (Combi),2,Absent,No,Free,From the real estate office,...,False,False,False,False,False,False,False,False,371489.177489,1
3,395000,143,124.0,5,Underfloor heating,2,Absent,No,Free,From the construction company,...,False,False,False,False,False,False,False,False,374425.020979,1
4,485000,139,115.0,9,Center (Share Meter),2,Available,No,Property owner,From the real estate office,...,False,False,False,True,False,False,False,False,374425.020979,1
5,305000,100,85.0,5,Natural Gas (Combi),1,Available,No,Tenant,From the real estate office,...,False,False,False,False,True,False,False,False,383901.222222,1
6,545000,130,105.0,4,Natural Gas (Combi),1,Available,No,Free,From the real estate office,...,False,False,False,False,False,False,False,False,405344.859060,1
7,190000,160,140.0,5,Natural Gas (Combi),1,Available,No,Free,From the real estate office,...,False,False,False,False,False,False,False,False,176000.000000,0
8,1850000,90,85.0,6,Natural Gas (Combi),1,Available,No,Free,From the real estate office,...,False,False,False,False,False,False,False,False,653862.903226,1
9,800000,140,120.0,8,Natural Gas (Combi),2,Available,No,Property owner,From the real estate office,...,False,False,False,False,False,False,False,False,454382.105691,1


In [ ]:
# Bu hücreyi df.head(10) hücresinden SONRA ekleyin

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler yüklendi!")
print(f"Mevcut DataFrame boyutu: {df.shape}")

Kütüphaneler yüklendi!
Mevcut DataFrame boyutu: (27381, 220)


In [ ]:
# Hedef değişkeni ve sayısal sütunları belirle
target_col = 'Log_Price'
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['Price', 'Log_Price']]

print(f"İşlenecek sayısal sütun sayısı: {len(numeric_cols)}")

# 1. POLİNOMİYAL ÖZELLİKLER (En önemli 5 feature için)
key_features = ['m² (Gross)', 'm² (Net)', 'Total_Rooms', 'Building_Age_Numeric', 'Floor_Numeric']
for col in key_features:
    if col in df.columns:
        df[f'{col}_squared'] = df[col] ** 2
        df[f'{col}_sqrt'] = np.sqrt(np.abs(df[col]) + 1)
        df[f'{col}_log'] = np.log1p(np.abs(df[col]))

print("Polinomiyal özellikler eklendi.")

İşlenecek sayısal sütun sayısı: 172
Polinomiyal özellikler eklendi.


In [ ]:
# Number of floors sütununu sayısala çevir
if 'Number of floors' in df.columns:
    df['Number of floors'] = pd.to_numeric(df['Number of floors'], errors='coerce')
    df['Number of floors'] = df['Number of floors'].fillna(df['Number of floors'].median())

# Oda başına metrekare (Bu güvenli)
df['Sqm_per_room'] = df['m² (Net)'] / (df['Total_Rooms'] + 1)

# Bina yaşı ve kat etkileşimi (Güvenli)
df['Age_Floor_interaction'] = df['Building_Age_Numeric'] * df['Floor_Numeric']

# Lüks ve metrekare etkileşimi (Güvenli)
df['Luxury_Sqm'] = df['Luxury_Score'] * df['m² (Net)']

# Güvenlik ve ulaşım kombinasyonu (Güvenli)
df['Security_Transport'] = df['Security_Index'] * df['Transport_Score']

# Toplam özellik skoru (Güvenli)
df['Total_Feature_Score'] = df['Luxury_Score'] + df['Security_Index'] + df['Transport_Score'] + df['View_Score']

# Kat / Toplam kat oranı (Güvenli)
df['Floor_Ratio'] = df['Floor_Numeric'] / (df['Number of floors'] + 1)

print("Etkileşim özellikleri eklendi (Price leakage temizlendi).")

Etkileşim özellikleri eklendi (Price leakage temizlendi).


In [ ]:
# Sadece binary (0/1) sütunları seç
binary_cols = [col for col in df.columns if df[col].dropna().isin([0, 1]).all() and col not in ['Price', 'Log_Price']]

# Toplam "evet" sayısı (ev özelliklerinin zenginliği)
df['Total_Yes_Count'] = df[binary_cols].sum(axis=1)

# Oransal özellik yoğunluğu
df['Feature_Density'] = df['Total_Yes_Count'] / len(binary_cols)

print(f"Binary sütun sayısı: {len(binary_cols)}")
print("İstatistiksel özellikler eklendi.")

Binary sütun sayısı: 199
İstatistiksel özellikler eklendi.


In [ ]:
# Kümeleme için kullanılacak temel özellikler
cluster_features = ['m² (Gross)', 'm² (Net)', 'Total_Rooms', 'Building_Age_Numeric', 
                    'Floor_Numeric', 'Luxury_Score', 'Security_Index', 'Transport_Score']
cluster_features = [c for c in cluster_features if c in df.columns]

X_cluster = df[cluster_features].fillna(df[cluster_features].median())

# Standardize
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# KMeans kümeleme
for n_clusters in [5, 10]:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df[f'Cluster_{n_clusters}'] = kmeans.fit_predict(X_cluster_scaled)
    
    # Cluster merkezine uzaklık
    distances = kmeans.transform(X_cluster_scaled)
    df[f'Cluster_{n_clusters}_min_dist'] = distances.min(axis=1)

print("Clustering özellikleri eklendi.")

Clustering özellikleri eklendi.


In [ ]:
# Tüm sayısal sütunları al (hedef hariç)
numeric_for_pca = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_for_pca = [c for c in numeric_for_pca if c not in ['Price', 'Log_Price']]

X_pca = df[numeric_for_pca].fillna(df[numeric_for_pca].median())

# Standardize
scaler_pca = StandardScaler()
X_pca_scaled = scaler_pca.fit_transform(X_pca)

# PCA
n_components = min(10, len(numeric_for_pca))
pca = PCA(n_components=n_components)
pca_features = pca.fit_transform(X_pca_scaled)

for i in range(n_components):
    df[f'PCA_{i+1}'] = pca_features[:, i]

print(f"PCA açıklanan varyans oranları: {pca.explained_variance_ratio_[:5]}")
print(f"Toplam açıklanan varyans: {sum(pca.explained_variance_ratio_):.2%}")

PCA açıklanan varyans oranları: [0.22280887 0.07834661 0.05929912 0.04504599 0.03569258]
Toplam açıklanan varyans: 55.18%


In [ ]:
# Inf değerleri NaN'a çevir
df = df.replace([np.inf, -np.inf], np.nan)

# NaN değerleri medyan ile doldur
numeric_cols_final = df.select_dtypes(include=[np.number]).columns
df[numeric_cols_final] = df[numeric_cols_final].fillna(df[numeric_cols_final].median())

print("NaN ve Inf değerler temizlendi.")
print(f"Final DataFrame boyutu: {df.shape}")
print(f"Eksik değer sayısı: {df.isnull().sum().sum()}")

NaN ve Inf değerler temizlendi.
Final DataFrame boyutu: (27381, 261)
Eksik değer sayısı: 0


In [ ]:
# Hedef değişken
y = df['Log_Price']
X = df.drop(columns=['Price', 'Log_Price'])

# Sadece sayısal sütunları al
X_numeric = X.select_dtypes(include=[np.number])

print(f"Toplam feature sayısı: {X_numeric.shape[1]}")

# Correlation bazlı seçim
correlations = X_numeric.corrwith(y).abs().sort_values(ascending=False)
print("\nEn yüksek korelasyonlu 20 feature:")
print(correlations.head(20))

# Random Forest importance
rf_selector = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_selector.fit(X_numeric.fillna(0), y)
importances = pd.Series(rf_selector.feature_importances_, index=X_numeric.columns).sort_values(ascending=False)
print("\nRF Importance - En önemli 20 feature:")
print(importances.head(20))

Toplam feature sayısı: 214

En yüksek korelasyonlu 20 feature:
Price_per_sqm_gross           0.808634
Price_per_sqm_net             0.753777
Neighborhood_TE_Log           0.703945
PCA_3                         0.678163
Neighborhood_TargetEncoded    0.670636
Neighborhood_TE_Normalized    0.670636
m² (Gross)_log                0.604721
m² (Gross)_sqrt               0.586110
m² (Net)_sqrt                 0.555638
m² (Net)                      0.554069
m² (Net)_log                  0.541718
m² (Net)_squared              0.503611
Total_Rooms_log               0.479599
Total_Rooms_sqrt              0.478341
Total_Rooms                   0.472494
Sqm_per_room                  0.455638
Total_Rooms_squared           0.440000
Cluster_10_min_dist           0.400488
m² (Gross)                    0.397664
Air conditioning              0.359811
dtype: float64

RF Importance - En önemli 20 feature:
Price_per_sqm_gross    0.701735
PCA_3                  0.117294
m² (Gross)             0.042698
m² (Gro

In [ ]:
# Final dosyayı kaydet
output_filename = 'hackathon_ultra_optimized.csv'
df.to_csv(output_filename, index=False)

print(f"=" * 50)
print(f"İşlemler tamamlandı!")
print(f"Dosya kaydedildi: {output_filename}")
print(f"Final Veri Seti Boyutu: {df.shape}")
print(f"=" * 50)
print("\nModel Eğitimi İçin Notlar:")
print("- Hedef Değişken (y): 'Log_Price'")
print("- X: 'Price' ve 'Log_Price' hariç tüm sayısal sütunlar")
print("- Tahmin sonrası: np.expm1(prediction) ile gerçek fiyata dönün")

İşlemler tamamlandı!
Dosya kaydedildi: hackathon_ultra_optimized.csv
Final Veri Seti Boyutu: (27381, 261)

Model Eğitimi İçin Notlar:
- Hedef Değişken (y): 'Log_Price'
- X: 'Price' ve 'Log_Price' hariç tüm sayısal sütunlar
- Tahmin sonrası: np.expm1(prediction) ile gerçek fiyata dönün


In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# Leaky sütunları kaldır
leaky_cols = ['Price_per_sqm_gross', 'Price_per_sqm_net', 
              'Neighborhood_TargetEncoded', 'Neighborhood_TE_Normalized', 
              'Neighborhood_TE_Log']
leaky_cols = [c for c in leaky_cols if c in df.columns]

# Veriyi hazırla
y = df['Log_Price']
X = df.drop(columns=['Price', 'Log_Price'] + leaky_cols).select_dtypes(include=[np.number])
X = X.fillna(X.median())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelleri test et
models = {
    'Random Forest': RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
    'Extra Trees': ExtraTreesRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
}

print("Model Performansları (Leakage Temizlenmiş):")
print("-" * 50)

for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_r2 = r2_score(y_test, y_pred)
    
    print(f"{name}:")
    print(f"  CV R² Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Test R² Score: {test_r2:.4f}")
    print()

Model Performansları (Leakage Temizlenmiş):
--------------------------------------------------
Random Forest:
  CV R² Score: 0.8215 (+/- 0.0105)
  Test R² Score: 0.8297

Gradient Boosting:
  CV R² Score: 0.8630 (+/- 0.0078)
  Test R² Score: 0.8691

Extra Trees:
  CV R² Score: 0.8114 (+/- 0.0068)
  Test R² Score: 0.8198

